# Advanced Regression House Sales

## 1. Setting up workspace

In [2]:
library("AzureML")
ws <- workspace(id='d4e707d1a7c240be9b71a52626f9afba',
               auth='4COx173QNxC+4NWxrABe78hCmosr0QZ/+x5ITFsxaDiVgkmLvYqP2Ew/D+6+rjdApDT1/KPGWaLHjnhNqO0PfQ==')

## 2. Loading libraries

In [3]:
library('ggplot2')
library('ggthemes') 
library('scales')
library('dplyr') 
library('mice')
library('randomForest') 
library('data.table')
library('gridExtra')
library('corrplot') 
library('GGally')
library('e1071')


Attaching package: âdplyrâ

The following objects are masked from âpackage:statsâ:

    filter, lag

The following objects are masked from âpackage:baseâ:

    intersect, setdiff, setequal, union

Loading required package: lattice
randomForest 4.6-12
Type rfNews() to see new features/changes/bug fixes.

Attaching package: ârandomForestâ

The following object is masked from âpackage:dplyrâ:

    combine

The following object is masked from âpackage:ggplot2â:

    margin


Attaching package: âdata.tableâ

The following objects are masked from âpackage:dplyrâ:

    between, first, last


Attaching package: âgridExtraâ

The following object is masked from âpackage:randomForestâ:

    combine

The following object is masked from âpackage:dplyrâ:

    combine

corrplot 0.84 loaded

Attaching package: âGGallyâ

The following object is masked from âpackage:dplyrâ:

    nasa



## 3. Loading and combining datasets

In [10]:
train <- download.datasets(ws, "train.csv")
test <- download.datasets(ws, "test.csv")

#creating saleprice variable in test data
test$SalePrice<-rep(NA,1459)
house<-bind_rows(train,test)

## 4. Data Exploration

### Univariate Approach

In [11]:
#general summary
dim(house)
summary(house)

#show freq counts for all variables where <20 categories in the variable
lapply(house,
       function(var)
         {
          if(dim(table(var))<20) table(var)#if there <50 items then show table
         }
  )

[1] 2919   81

       Id           MSSubClass       MSZoning          LotFrontage    
 Min.   :   1.0   Min.   : 20.00   Length:2919        Min.   : 21.00  
 1st Qu.: 730.5   1st Qu.: 20.00   Class :character   1st Qu.: 59.00  
 Median :1460.0   Median : 50.00   Mode  :character   Median : 68.00  
 Mean   :1460.0   Mean   : 57.14                      Mean   : 69.31  
 3rd Qu.:2189.5   3rd Qu.: 70.00                      3rd Qu.: 80.00  
 Max.   :2919.0   Max.   :190.00                      Max.   :313.00  
                                                      NA's   :486     
    LotArea          Street             Alley             LotShape        
 Min.   :  1300   Length:2919        Length:2919        Length:2919       
 1st Qu.:  7478   Class :character   Class :character   Class :character  
 Median :  9453   Mode  :character   Mode  :character   Mode  :character  
 Mean   : 10168                                                           
 3rd Qu.: 11570                                          

$Id
NULL

$MSSubClass
var
  20   30   40   45   50   60   70   75   80   85   90  120  150  160  180  190 
1079  139    6   18  287  575  128   23  118   48  109  182    1  128   17   61 

$MSZoning
var
C (all)      FV      RH      RL      RM 
     25     139      26    2265     460 

$LotFrontage
NULL

$LotArea
NULL

$Street
var
Grvl Pave 
  12 2907 

$Alley
var
Grvl Pave 
 120   78 

$LotShape
var
 IR1  IR2  IR3  Reg 
 968   76   16 1859 

$LandContour
var
 Bnk  HLS  Low  Lvl 
 117  120   60 2622 

$Utilities
var
AllPub NoSeWa 
  2916      1 

$LotConfig
var
 Corner CulDSac     FR2     FR3  Inside 
    511     176      85      14    2133 

$LandSlope
var
 Gtl  Mod  Sev 
2778  125   16 

$Neighborhood
NULL

$Condition1
var
Artery  Feedr   Norm   PosA   PosN   RRAe   RRAn   RRNe   RRNn 
    92    164   2511     20     39     28     50      6      9 

$Condition2
var
Artery  Feedr   Norm   PosA   PosN   RRAe   RRAn   RRNn 
     5     13   2889      4      4      1      1      2 

$BldgT

##### Plots on Power BI

In [12]:
cat_var <- names(train)[which(sapply(train, is.character))]
cat_car <- c(cat_var, 'BedroomAbvGr', 'HalfBath', ' KitchenAbvGr','BsmtFullBath', 'BsmtHalfBath', 'MSSubClass')
numeric_var <- names(train)[which(sapply(train, is.numeric))]

## Creating one training dataset with categorical variable and one with numeric variable. 
## We will use this for data visualization.
train1_cat<-train[cat_var]
train1_num<-train[numeric_var]

## Bar plot/Density plot function
## Bar plot function
plotHist <- function(data_in, i) 
{
  data <- data.frame(x=data_in[[i]])
  p <- ggplot(data=data, aes(x=factor(x))) + stat_count() + xlab(colnames(data_in)[i]) + theme_light() + 
    theme(axis.text.x = element_text(angle = 90, hjust =1))
  return (p)
}

## Density plot function
plotDen <- function(data_in, i){
  data <- data.frame(x=data_in[[i]], SalePrice = data_in$SalePrice)
  p <- ggplot(data= data) + geom_line(aes(x = x), stat = 'density', size = 1,alpha = 1.0) +
    xlab(paste0((colnames(data_in)[i]), '\n', 'Skewness: ',round(skewness(data_in[[i]], na.rm = TRUE), 2))) + theme_light() 
  return(p)
  
}
## Function to call both Bar plot and Density plot function
doPlots <- function(data_in, fun, ii, ncol=3) 
{
  pp <- list()
  for (i in ii) {
    p <- fun(data_in=data_in, i=i)
    pp <- c(pp, list(p))
  }
  do.call("grid.arrange", c(pp, ncol=ncol))
}

## Barplots for the categorical features
doPlots(train1_cat, fun = plotHist, ii = 1:4, ncol = 2)

In [13]:
doPlots(train1_cat, fun = plotHist, ii  = 5:8, ncol = 2)

In [14]:
doPlots(train1_cat, fun = plotHist, ii = 9:12, ncol = 2)

In [15]:
doPlots(train1_cat, fun = plotHist, ii = 13:18, ncol = 2)

In [16]:
doPlots(train1_cat, fun = plotHist, ii = 19:22, ncol = 2)

In [26]:
ggplot(train, aes(x = Neighborhood, y = SalePrice, hjust=10)) +
  geom_boxplot() +
  geom_hline(aes(yintercept=80), 
             colour='red', linetype='dashed', lwd=2) +
  scale_y_continuous(labels=dollar_format()) +
  theme_few() 

Conclusion: boxplot between the neighboorhoods and sale price shows that BrookSide and South & West of Iowa State University have cheap houses. While Northridge and Northridge Heights are rich neighborhoods with several outliers in terms of price.

In [27]:
#Density plots for numeric variables
doPlots(train1_num, fun = plotDen, ii = 2:6, ncol = 2)

Warning message:
âRemoved 259 rows containing non-finite values (stat_density).â

In [28]:
doPlots(train1_num, fun = plotDen, ii = 7:12, ncol = 2)

Warning message:
âRemoved 8 rows containing non-finite values (stat_density).â

In [29]:
doPlots(train1_num, fun = plotDen, ii = 13:17, ncol = 2)

Conclusion: Density plots of the features indicates that the features are skewed. The denisty plot for YearBuilt shows that the data set contains a mix of new and old houses. It shows a downturn in the number of houses in recent years, possibily due to the housing crisis.

In [30]:
#Histogram for few numeric variable
doPlots(train1_num, fun = plotHist, ii = 18:23, ncol = 2)

Conclusion: The histograms below show that majority of the houses have 2 full baths, 0 half baths, and have an average of 3 bedrooms.

In [46]:
#Explore the correlation
par(pin=c(.1,.1))
correlations <- cor(na.omit(train1_num[,-1]))
row_indic <- apply(correlations, 1, function(x) sum(x > 0.3 | x < -0.3) > 1)
correlations<- correlations[row_indic ,row_indic ]
corrplot(correlations, method="square")

In [47]:
## Histogram for target variable
hist(train$SalePrice)
## Conclusion: From Histogram, we could see that it deviates from normal distribution and has positive skewness.

#### Data Cleaning

In [48]:
# Plotting 'GrLivArea' too see if there are any outliers
ggplot(train,aes(y=SalePrice,x=GrLivArea))+geom_point()

In [49]:
summary(train$GrLivArea)

   Min. 1st Qu.  Median    Mean 3rd Qu.    Max. 
    334    1130    1464    1515    1777    5642 

In [50]:
# There are outliers in 'GrLivArea' field. Let's remove those outliers.
train <- train[train$GrLivArea<=4000,]

In [51]:
## To find number of missing value for all variable in train dataset
#colSums(sapply(train, is.na))
# To find number of missing value for all variable in combined dataset (Train+Test)
sapply(house[,1:80], function(x) sum(is.na(x)))

Id    MSSubClass      MSZoning   LotFrontage       LotArea 
            0             0             4           486             0 
       Street         Alley      LotShape   LandContour     Utilities 
            0          2721             0             0             2 
    LotConfig     LandSlope  Neighborhood    Condition1    Condition2 
            0             0             0             0             0 
     BldgType    HouseStyle   OverallQual   OverallCond     YearBuilt 
            0             0             0             0             0 
 YearRemodAdd     RoofStyle      RoofMatl   Exterior1st   Exterior2nd 
            0             0             0             1             1 
   MasVnrType    MasVnrArea     ExterQual     ExterCond    Foundation 
           24            23             0             0             0 
     BsmtQual      BsmtCond  BsmtExposure  BsmtFinType1    BsmtFinSF1 
           81            82            82            79             1 
 BsmtFinType2    BsmtFinSF2     BsmtUnfSF   TotalBsmtSF       Heating 
           80             1             1             1             0 
    HeatingQC    CentralAir    Electrical     X1stFlrSF     X2ndFlrSF 
            0             0             1             0             0 
 LowQualFinSF     GrLivArea  BsmtFullBath  BsmtHalfBath      FullBath 
            0             0             2             2             0 
     HalfBath  BedroomAbvGr  KitchenAbvGr   KitchenQual  TotRmsAbvGrd 
            0             0             0             1             0 
   Functional    Fireplaces   FireplaceQu    GarageType   GarageYrBlt 
            2             0          1420           157           159 
 GarageFinish    GarageCars    GarageArea    GarageQual    GarageCond 
          159             1             1           159           159 
   PavedDrive    WoodDeckSF   OpenPorchSF EnclosedPorch    X3SsnPorch 
            0             0             0             0             0 
  ScreenPorch      PoolArea        PoolQC         Fence   MiscFeature 
            0             0          2909          2348          2814 
      MiscVal        MoSold        YrSold      SaleType SaleCondition 
            0             0             0             1             0

In [52]:
## Taking all the missing data indices in one variables
Missing_indices <- sapply(train,function(x) sum(is.na(x)))
Missing_Summary <- data.frame(index = names(train),Missing_Values=Missing_indices)
Missing_Summary[Missing_Summary$Missing_Values > 0,]

,index,Missing_Values
LotFrontage,LotFrontage,259
Alley,Alley,1365
MasVnrType,MasVnrType,8
MasVnrArea,MasVnrArea,8
BsmtQual,BsmtQual,37
BsmtCond,BsmtCond,37
BsmtExposure,BsmtExposure,38
BsmtFinType1,BsmtFinType1,37
BsmtFinType2,BsmtFinType2,38
Electrical,Electrical,1


In [53]:
#Combining train and test data for quicker data prep
test$SalePrice <- NA
train$isTrain <- 1
test$isTrain <- 0
house <- rbind(train,test)

In [55]:
#MasVnrArea
house$MasVnrArea[which(is.na(house$MasVnrArea))] <- mean(house$MasVnrArea,na.rm=T)
#Alley
house$Alley1 <- as.character(house$Alley)
house$Alley1[which(is.na(house$Alley))] <- "None"
#table(house$Alley1)
house$Alley <- as.factor(house$Alley1)
house <- subset(house,select = -Alley1)
#MasVnrType
house$MasVnrType1 <- as.character(house$MasVnrType)
house$MasVnrType1[which(is.na(house$MasVnrType))] <- "None"
house$MasVnrType <- as.factor(house$MasVnrType1)
house <- subset(house,select = -MasVnrType1)
#table(house$MasVnrType)
#Lot Frontage
house$LotFrontage[which(is.na(house$LotFrontage))] <- median(house$LotFrontage,na.rm = T)
#Fireplace QU
house$FireplaceQu1 <- as.character(house$FireplaceQu)
house$FireplaceQu1[which(is.na(house$FireplaceQu))] <- "None"
house$FireplaceQu <- as.factor(house$FireplaceQu1)
house <- subset(house,select = -FireplaceQu1)
#Pool QC
house$PoolQC1 <- as.character(house$PoolQC)
house$PoolQC1[which(is.na(house$PoolQC))] <- "None"
house$PoolQC <- as.factor(house$PoolQC1)
house <- subset(house,select = -PoolQC1)
#Fence
house$Fence1 <- as.character(house$Fence)
house$Fence1[which(is.na(house$Fence))] <- "None"
house$Fence <- as.factor(house$Fence1)
house <- subset(house,select = -Fence1)
#Misc Feature
house$MiscFeature1 <- as.character(house$MiscFeature)
house$MiscFeature1[which(is.na(house$MiscFeature))] <- "None"
house$MiscFeature <- as.factor(house$MiscFeature1)
house <- subset(house,select = -MiscFeature1)
#Garage Type
house$GarageType1 <- as.character(house$GarageType)
house$GarageType1[which(is.na(house$GarageType))] <- "None"
house$GarageType <- as.factor(house$GarageType1)
house <- subset(house,select = -GarageType1)
#GarageYrBlt
house$GarageYrBlt[which(is.na(house$GarageYrBlt))] <- 0 
#GarageFinish
house$GarageFinish1 <- as.character(house$GarageFinish)
house$GarageFinish1[which(is.na(house$GarageFinish))] <- "None"
house$GarageFinish <- as.factor(house$GarageFinish1)
house <- subset(house,select = -GarageFinish1)
#GarageQual
house$GarageQual1 <- as.character(house$GarageQual)
house$GarageQual1[which(is.na(house$GarageQual))] <- "None"
house$GarageQual <- as.factor(house$GarageQual1)
house <- subset(house,select = -GarageQual1)
#GarageCond
house$GarageCond1 <- as.character(house$GarageCond)
house$GarageCond1[which(is.na(house$GarageCond))] <- "None"
house$GarageCond <- as.factor(house$GarageCond1)
house <- subset(house,select = -GarageCond1)
#BsmtQual
house$BsmtQual1 <- as.character(house$BsmtQual)
house$BsmtQual1[which(is.na(house$BsmtQual))] <- "None"
house$BsmtQual <- as.factor(house$BsmtQual1)
house <- subset(house,select = -BsmtQual1)
#BsmtCond
house$BsmtCond1 <- as.character(house$BsmtCond)
house$BsmtCond1[which(is.na(house$BsmtCond))] <- "None"
house$BsmtCond <- as.factor(house$BsmtCond1)
house <- subset(house,select = -BsmtCond1)
#BsmtExposure
house$BsmtExposure1 <- as.character(house$BsmtExposure)
house$BsmtExposure1[which(is.na(house$BsmtExposure))] <- "None"
house$BsmtExposure <- as.factor(house$BsmtExposure1)
house <- subset(house,select = -BsmtExposure1)
#BsmtFinType1
house$BsmtFinType11 <- as.character(house$BsmtFinType1)
house$BsmtFinType11[which(is.na(house$BsmtFinType1))] <- "None"
house$BsmtFinType1 <- as.factor(house$BsmtFinType11)
house <- subset(house,select = -BsmtFinType11)
#BsmtFinType2
house$BsmtFinType21 <- as.character(house$BsmtFinType2)
house$BsmtFinType21[which(is.na(house$BsmtFinType2))] <- "None"
house$BsmtFinType2 <- as.factor(house$BsmtFinType21)
house <- subset(house,select = -BsmtFinType21)
#Electrical
house$Electrical1 <- as.character(house$Electrical)
house$Electrical1[which(is.na(house$Electrical))] <- "None"
house$Electrical <- as.factor(house$Electrical1)
house <- subset(house,select = -Electrical1)

In [56]:
#Factorizing
house$MSZoning<- factor(house$MSZoning)
house$Street <- factor(house$Street)
house$LotShape <-factor(house$LotShape )
house$LandContour<-factor(house$LandContour)
house$Utilities<-factor(house$Utilities)
house$LotConfig<-factor(house$LotConfig)
house$LandSlope<-factor(house$LandSlope)
house$Neighborhood<-factor(house$Neighborhood)
house$Condition1<-factor(house$Condition1)
house$Condition2<-factor(house$Condition2)
house$BldgType<-factor(house$BldgType)
house$HouseStyle<-factor(house$HouseStyle)
house$RoofStyle<-factor(house$RoofStyle)
house$RoofMatl<-factor(house$RoofMatl)
house$Exterior1st<-factor(house$Exterior1st)
house$Exterior2nd<-factor(house$Exterior2nd)
house$ExterQual<-factor(house$ExterQual)
house$ExterCond<-factor(house$ExterCond)
house$Foundation<-factor(house$Foundation)
house$Heating<-factor(house$Heating)
house$HeatingQC<-factor(house$HeatingQC)
house$CentralAir<-factor(house$CentralAir)
house$KitchenQual<-factor(house$KitchenQual)
house$Functional<-factor(house$Functional)
house$PavedDrive<-factor(house$PavedDrive)
house$SaleType<-factor(house$SaleType)
house$SaleCondition<-factor(house$SaleCondition)
str(house)

'data.frame':	2915 obs. of  82 variables:
 $ Id           : int  1 2 3 4 5 6 7 8 9 10 ...
 $ MSSubClass   : int  60 20 60 70 60 50 20 60 50 190 ...
 $ MSZoning     : Factor w/ 5 levels "C (all)","FV",..: 4 4 4 4 4 4 4 4 5 4 ...
 $ LotFrontage  : int  65 80 68 60 84 85 75 68 51 50 ...
 $ LotArea      : int  8450 9600 11250 9550 14260 14115 10084 10382 6120 7420 ...
 $ Street       : Factor w/ 2 levels "Grvl","Pave": 2 2 2 2 2 2 2 2 2 2 ...
 $ Alley        : Factor w/ 3 levels "Grvl","None",..: 2 2 2 2 2 2 2 2 2 2 ...
 $ LotShape     : Factor w/ 4 levels "IR1","IR2","IR3",..: 4 4 1 1 1 1 4 1 4 4 ...
 $ LandContour  : Factor w/ 4 levels "Bnk","HLS","Low",..: 4 4 4 4 4 4 4 4 4 4 ...
 $ Utilities    : Factor w/ 2 levels "AllPub","NoSeWa": 1 1 1 1 1 1 1 1 1 1 ...
 $ LotConfig    : Factor w/ 5 levels "Corner","CulDSac",..: 5 3 5 1 3 5 5 1 5 1 ...
 $ LandSlope    : Factor w/ 3 levels "Gtl","Mod","Sev": 1 1 1 1 1 1 1 1 1 1 ...
 $ Neighborhood : Factor w/ 25 levels "Blmngtn","Blueste",..: 6 25 6

In [57]:
#Taking all the column classes in one variable so as to seperate factors from numerical variables.
Column_classes <- sapply(names(house),function(x){class(house[[x]])})
numeric_columns <-names(Column_classes[Column_classes != "factor"])

#determining skew of each numeric variable
skew <- sapply(numeric_columns,function(x){skewness(house[[x]],na.rm = T)})

# Let us determine a threshold skewness and transform all variables above the treshold.
skew <- skew[skew > 0.75]

# transform excessively skewed features with log(x + 1)
for(x in names(skew)) 
{
  house[[x]] <- log(house[[x]] + 1)
}

### Multivariate Approach (random forest)

In [58]:
# create train and test datasets
train <- house[house$isTrain==1,]
test <- house[house$isTrain==0,]
smp_size <- floor(0.75 * nrow(train))

## setting the seed to make the partition reproducible
set.seed(123)
train_ind <- sample(seq_len(nrow(train)), size = smp_size)

train_new <- train[train_ind, ]
validate <- train[-train_ind, ]
train_new <- subset(train_new,select=-c(Id,isTrain))
validate <- subset(validate,select=-c(Id,isTrain))
nrow(train_new)

[1] 1092

In [59]:
nrow(validate)

[1] 364

In [98]:
#Building the mode
library(randomForest)
house_model <- randomForest(SalePrice~.,
                            data = train_new, do.trace=T)

     |      Out-of-bag   |
Tree |      MSE  %Var(y) |
   1 |  0.05844    36.58 |
   2 |  0.05615    35.15 |
   3 |  0.05138    32.16 |
   4 |  0.04553    28.50 |
   5 |  0.04422    27.68 |
   6 |  0.03918    24.53 |
   7 |  0.03638    22.78 |
   8 |  0.03414    21.37 |
   9 |  0.03282    20.54 |
  10 |  0.03091    19.35 |
  11 |  0.02974    18.61 |
  12 |  0.02886    18.07 |
  13 |  0.02809    17.58 |
  14 |   0.0275    17.22 |
  15 |  0.02713    16.98 |
  16 |  0.02676    16.75 |
  17 |  0.02567    16.07 |
  18 |  0.02563    16.04 |
  19 |  0.02548    15.95 |
  20 |  0.02486    15.56 |
  21 |  0.02476    15.50 |
  22 |  0.02453    15.35 |
  23 |  0.02431    15.22 |
  24 |  0.02395    14.99 |
  25 |  0.02395    14.99 |
  26 |  0.02372    14.85 |
  27 |  0.02367    14.82 |
  28 |  0.02333    14.60 |
  29 |  0.02329    14.58 |
  30 |  0.02334    14.61 |
  31 |  0.02313    14.48 |
  32 |   0.0231    14.46 |
  33 |  0.02292    14.35 |
  34 |  0.02275    14.24 |
  35 |   0.0226    14.15 |
 

In [100]:
house_model$confusion[, 'class.error']

NULL

In [101]:
mdl_rf1<-cforest(SalePrice~.,
             data=train_new,
             control=cforest_unbiased(ntree=100, mtry=8, minsplit=30,
                                      minbucket=10))

ERROR: Error in cforest(SalePrice ~ ., data = train_new, control = cforest_unbiased(ntree = 100, : could not find function "cforest"


In [93]:
#Variable importance
#dev.new(width=5, height=4)
importance    <- importance(house_model)
varImpPlot(house_model)

In [66]:
#Final prediction
#Predict using the test set
prediction <- predict(house_model,test)

In [97]:
install.packages('e1071', dependencies=TRUE)
install.packages("src/caret_6.0-68.zip", lib=".", repos= NULL, verbose=TRUE)
library("caret", lib.loc=".", verbose=TRUE)

Installing package into â/home/nbuser/Râ
(as âlibâ is unspecified)
system (cmd0): /opt/microsoft/ropen/3.4.1/lib64/R/bin/R CMD INSTALL
Warning message in install.packages("src/caret_6.0-68.zip", lib = ".", repos = NULL, :
âinstallation of package âsrc/caret_6.0-68.zipâ had non-zero exit statusâ

ERROR: Error in library("caret", lib.loc = ".", verbose = TRUE): there is no package called âcaretâ


In [70]:
#define log rmse function
rmse<-function(x,y){mean((log(x)-log(y))^2)^.5}

#define assess function
score_rf1<-function(df){
  preds_rf1<-predict(train_new,df,OOB=TRUE,type='response')
  sc1<<-data.frame(cbind(df,preds_rf1))
  names(sc1)[names(sc1)=='SalePrice.1']<<-'preds_rf1'
  rmse(sc1$SalePrice,sc1$preds_rf1)
}

#gauge accuracy
score_rf1(train_new)#in-sample observations
score_rf1(test)#out of sample observations

ERROR: Error in library(caret): there is no package called âcaretâ


In [ ]:
###7. Train a Linear Model

##Define training data

#cut back the modified bs data to include only training observations
ktrain_id<-subset(ktrain,select='Id')
lm_tr<-merge(bs,ktrain_id,by='Id')
lm_tr$log_SalePrice=log(lm_tr$SalePrice)

#subset to include numerics only
nums_use<-names(lm_tr)[sapply(lm_tr,is.numeric)]
nums_use=c('LotArea','OverallQual','OverallCond','BsmtFinSF1','BsmtUnfSF',
           'X1stFlrSF','X2ndFlrSF','Fireplaces','ScreenPorch',
           'Neighborhood_cg','KitchenQual_cg','nbhdsize_cg','LotFrontage',
           'BsmtFinSF2','BsmtQual_cg','log_SalePrice')

#highly correlated features
library(caret)
write.csv(cor(lm_tr[,nums_use]),file='cor.csv')
#<--view correlations in excel-->
findCorrelation(cor(lm_tr[,nums_use]))
findLinearCombos(lm_tr[,nums_use])
lm_tr<-subset(lm_tr,select=nums_use)

#split further into train/test for true out of sample test
set.seed(1)
tr_index<-sample(nrow(lm_tr),size=floor(.6*nrow(lm_tr)))
lm_tr_train<-lm_tr[tr_index,]
lm_tr_test<-lm_tr[-tr_index,]


##Train a linear model
mdl_lm<-lm(log_SalePrice~.,data=lm_tr_train)
summary(mdl_lm)#view initial results



###8. Score the Linear Model

#define log rmse function
rmse<-function(x,y){mean((log(x)-log(y))^2)^.5}

#define assess function
score_lm<-function(df){
  preds<-predict(mdl_lm,df,type='response')
  preds<-exp(preds)
  df$SalePrice=exp(df$log_SalePrice)
  sc2<<-data.frame(cbind(df,preds))
  rmse(sc2$SalePrice,sc2$preds)
}

#gauge accuracy
score_lm(lm_tr_train)#in-sample observations
score_lm(lm_tr_test)#out of sample observations